# ParabolaPyTorch
Extra required packages:
- matplotlib

In [ ]:
import gurobipy as gp
import torch
from gurobipy import GRB
from matplotlib import pyplot as plt
from gurobi_ml import add_predictor_constr

In [ ]:
x = torch.unsqueeze(torch.arange(-1, 1, 0.005), 1)

z = (x) ** 2

In [ ]:
plt.plot(x.data.numpy(), z.data.numpy())

plt.show()

In [ ]:
hidden_size = 28
# Define a simple sequential network
model = torch.nn.Sequential(
    torch.nn.Linear(1, 15),
    torch.nn.ReLU(),
    torch.nn.Linear(15, 15),
    torch.nn.ReLU(),
    torch.nn.Linear(15, 1),
)

In [ ]:
# Construct our loss function and an Optimizer.
criterion = torch.nn.MSELoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.2)

for t in range(10000):
    # Zero gradients
    optimizer.zero_grad()
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x.reshape(-1, 1))

    # Compute and print loss
    loss = criterion(y_pred, z)
    if t % 500 == 0:
        print(f"iteration {t} loss: {loss.item()}")

    if loss.item() < 1e-4:
        break
    # Backward pass in network to compute gradients
    loss.backward()
    # Update weights
    optimizer.step()
else:
    print(f"iteration {t} loss: {loss.item()}")

In [ ]:
y = model(x.reshape(-1, 1))
plt.plot(x.data.numpy(), y.data.numpy())

plt.show()

In [ ]:
y.min()

### Do the optimization model

In [ ]:
npoints = 1

In [ ]:
m = gp.Model()

x = m.addMVar((npoints, 1), lb=-1, ub=1, name="x")
y = m.addMVar((npoints, 1), lb=-GRB.INFINITY, name="y")

m.setObjective(y.sum(), gp.GRB.MINIMIZE)

In [ ]:
pred_constr = add_predictor_constr(m, model, x, y)

In [ ]:
pred_constr.print_stats()

### Finally optimize it

In [ ]:
m.Params.TimeLimit = 10
m.Params.MIPGap = 0.01

In [ ]:
m.optimize()

### Look at the solution

In [ ]:
x.X

In [ ]:
y.X

Copyright © 2023-2026 Gurobi Optimization, LLC